In [1]:
import os
%pwd

'h:\\Projects\\TextSummarizer\\research'

In [2]:
os.chdir("../")
%pwd

'h:\\Projects\\TextSummarizer'

In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: str
    tokenizer_path: str
    metric_file_path: str

In [17]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        return ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_path=config.metric_file_path
        )

In [18]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch
import pandas as pd
from tqdm import tqdm
from rouge_score import rouge_scorer

In [19]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]



    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,
                                batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu",
                                column_text="article",
                                column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        all_scores = {rn: [] for rn in metric.rouge_types}

        for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total=len(article_batches)):
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                            padding="max_length", return_tensors="pt")

            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device),
                            length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

            # Finally, we decode the generated texts,
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True)
                                                                for s in summaries]

            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


            #metric.add_batch(predictions=decoded_summaries, references=target_batch)

        #  Finally compute and return the ROUGE scores.
        # score = metric.compute()
        # return score

            for target, decoded_summary in zip(target_batch, decoded_summaries):
                scores = metric.score(target, decoded_summary)
                for rn in all_scores:
                    all_scores[rn].append(scores[rn])

        # Calculate the average scores
        final_scores = {}
        for rn in all_scores:
            # Instead of using sum, calculate the average for each score component separately
            if all_scores[rn]:
                precision_sum = sum([score.precision for score in all_scores[rn]])
                recall_sum = sum([score.recall for score in all_scores[rn]])
                fmeasure_sum = sum([score.fmeasure for score in all_scores[rn]])

                num_scores = len(all_scores[rn])
                final_scores[rn] = rouge_scorer.scoring.Score(
                    precision=precision_sum / num_scores,
                    recall=recall_sum / num_scores,
                    fmeasure=fmeasure_sum / num_scores
                )
            else:
                final_scores[rn] = rouge_scorer.scoring.Score(precision=0, recall=0, fmeasure=0)

        return final_scores
    
    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)

        test_dataset = load_from_disk(self.config.data_path)
        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        rouge_metric = rouge_scorer.RougeScorer(rouge_names, use_stemmer=True)

        score = self.calculate_metric_on_test_ds(
            test_dataset['test'][0:10], rouge_metric, model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary')

        rouge_dict = dict((rn, score[rn].fmeasure ) for rn in rouge_names )

        pd.DataFrame(rouge_dict, index = [f'pegasus'] ).to_csv(self.config.metric_file_path, index = False)

In [21]:
try:
    config_manager = ConfigurationManager()
    model_evaluation_config = config_manager.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.evaluate()
except Exception as e:
    raise e

[2024-11-22 15:29:52,973] 16 TextSummarizerLogger - INFO - yaml file config\config.yaml read successfully.
[2024-11-22 15:29:52,976] 16 TextSummarizerLogger - INFO - yaml file params.yaml read successfully.
[2024-11-22 15:29:52,977] 35 TextSummarizerLogger - INFO - Directory artifacts already exists.
[2024-11-22 15:29:52,979] 35 TextSummarizerLogger - INFO - Directory artifacts/model_evaluation already exists.
[2024-11-22 15:29:59,564] 83 absl - INFO - Using default tokenizer.


100%|██████████| 5/5 [05:08<00:00, 61.66s/it]
